## ASTR 119 Final Project Option #3
## Astronomical Source Detection

In [ ]:
import numpy as np
import sep

In [ ]:
# additional set up for reading the test image and displaying plots
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
#read image into standard 2-d numpy array

data = fits.getdata('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
data = data.byteswap(inplace=True).newbyteorder()

In [ ]:
#show our image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation = 'nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

## Background subtraction

In [ ]:
#measure a spatially varying background on the image
bkg = sep.Background(data)

In [ ]:
#get a 'global' mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as a 2-3 array
bkg_image = bkg.back()

In [ ]:
#show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#evaluate the background noise as 2-3 array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
#show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#subtract our background
data_sub = data - bkg

In [ ]:
objects = sep.extract(data_sub, 1.5, err = bkg.globalrms)

## Sources found in our detection

In [ ]:
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

#plot background - subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)

im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', norm=LogNorm(), origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]),
               width = 6*objects['a'][i],
               height = 6*objects['b'][i],
               angle=objects['theta'][i] * 180. /np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
hdu_list = fits.open('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
hdu_list.info()
image_data = hdu_list[0].data
header = hdu_list[0].header

In [ ]:
histogram = plt.hist(image_data.flatten(), bins='auto', range=(-.0019,.0019))


In [ ]:
#show the flux for the first 50 objects results:
for i in range(50):
    print('object {:d}: flux = {:f} +/- {:f}'.format(i, flux[i], fluxerr[i]))